In [5]:
import requests
import json
import mysql.connector
from datetime import datetime, timedelta
from tqdm import tqdm

# MySQL Configuration
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'root',
    'database': 'hft_analytics'
}

# Instrument token for example (341249 is for RELIANCE in this case)
INSTRUMENT_TOKEN = "341249"
USER_ID = "XST433"  # Update if needed

# Function to create MySQL table
def create_table():
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS historical_data_1min_hdfc (
            timestamp DATETIME PRIMARY KEY,
            open FLOAT,
            high FLOAT,
            low FLOAT,
            close FLOAT,
            volume INT,
            oi INT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Table created (if not exists)")

# Function to fetch and store data for a specific date
def fetch_and_store(date_str):
    url = f"https://kite.zerodha.com/oms/instruments/historical/{INSTRUMENT_TOKEN}/minute?user_id={USER_ID}&oi=1&from={date_str}&to={date_str}"
    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Authorization': 'enctoken HgCA1WIJ5p63kF2HQfCpELEIUs0YnLTlFnfHncKwKB0TqgYXKTq+KqZ/fIgJEF1g++MGHOizt3Nrx6/d0TUHnSjulAersUgj+cd+YSy6PJbv26ufV3qSUQ=='
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"❌ Failed for {date_str} — Status: {response.status_code}")
        return

    data = response.json()
    if data.get("status") != "success":
        print(f"❌ API error for {date_str} — {data}")
        return
    
    candles = data["data"]["candles"]
    if not candles:
        return
    
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()

    for candle in candles:
        timestamp, open_, high, low, close, volume, oi = candle
        timestamp = datetime.fromisoformat(timestamp.replace("+0530", "+05:30"))
        cursor.execute("""
            INSERT IGNORE INTO historical_data_1min_hdfc (timestamp, open, high, low, close, volume, oi)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (timestamp, open_, high, low, close, volume, oi))

    conn.commit()
    conn.close()
    print(f"✅ Data stored for {date_str}")

# Run for all dates from Jan 1, 2020 to April 30, 2025
start_date = datetime(2024 ,8, 21)
end_date = datetime(2025, 4, 30)

# Create the table first
create_table()

# Loop through all dates
date = start_date
while date <= end_date:
    fetch_and_store(date.strftime("%Y-%m-%d"))
    date += timedelta(days=1)


✅ Table created (if not exists)
✅ Data stored for 2024-08-21
✅ Data stored for 2024-08-22
✅ Data stored for 2024-08-23
✅ Data stored for 2024-08-26
✅ Data stored for 2024-08-27
✅ Data stored for 2024-08-28
✅ Data stored for 2024-08-29
✅ Data stored for 2024-08-30
✅ Data stored for 2024-09-02
✅ Data stored for 2024-09-03
✅ Data stored for 2024-09-04
✅ Data stored for 2024-09-05
✅ Data stored for 2024-09-06
✅ Data stored for 2024-09-09
✅ Data stored for 2024-09-10
✅ Data stored for 2024-09-11
✅ Data stored for 2024-09-12
✅ Data stored for 2024-09-13
✅ Data stored for 2024-09-16
✅ Data stored for 2024-09-17
✅ Data stored for 2024-09-18
✅ Data stored for 2024-09-19
✅ Data stored for 2024-09-20
✅ Data stored for 2024-09-23
✅ Data stored for 2024-09-24
✅ Data stored for 2024-09-25
✅ Data stored for 2024-09-26
✅ Data stored for 2024-09-27
✅ Data stored for 2024-09-30
✅ Data stored for 2024-10-01
✅ Data stored for 2024-10-03
✅ Data stored for 2024-10-04
✅ Data stored for 2024-10-07
✅ Data stor